In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("../data/cleaned_data.csv", parse_dates=['timestamp', 'signup_date'])
df = dataset.copy()
df.head()

,customer_id,location,gender,signup_date,sale_id,product_id,timestamp,quantity,price,revenue,name,category,supplier,customer_lifetime_value,timestamp_last,days_since_last_purchase
0,1,Ibadan,Male,2020-12-25,164,56,2023-09-08,4,8313,33252,Product_56,Groceries,Supplier A,692985,2023-09-08,114
1,1,Ibadan,Male,2020-12-25,424,28,2022-11-19,4,27160,108640,Product_28,Groceries,Supplier D,692985,2023-09-08,114
2,1,Ibadan,Male,2020-12-25,498,44,2021-11-30,2,24321,48642,Product_44,Books,Supplier B,692985,2023-09-08,114
3,1,Ibadan,Male,2020-12-25,586,41,2021-11-22,4,11883,47532,Product_41,Groceries,Supplier D,692985,2023-09-08,114
4,1,Ibadan,Male,2020-12-25,841,68,2021-10-08,1,14788,14788,Product_68,Electronics,Supplier A,692985,2023-09-08,114


In [2]:
df.dtypes

customer_id                          int64
location                            object
gender                              object
signup_date                 datetime64[ns]
sale_id                              int64
product_id                           int64
timestamp                   datetime64[ns]
quantity                             int64
price                                int64
revenue                              int64
name                                object
category                            object
supplier                            object
customer_lifetime_value              int64
timestamp_last                      object
days_since_last_purchase             int64
dtype: object

In [3]:
reference = df["timestamp"].max() + pd.Timedelta(days=1)
reference

Timestamp('2024-01-01 00:00:00')

In [4]:
columns = ["product_id", "customer_id", "timestamp", "revenue"]
df_dataset = df[columns]

In [5]:
rfm_dataset = df_dataset.groupby("customer_id").agg({
    "timestamp": lambda x: (reference - x.max()).days,
    "customer_id": "count",
    "revenue": "sum"
}).rename(columns={
    "timestamp": "Recency",
    "customer_id": "Frequency",
    "revenue": "Monetary"
})

In [6]:
rfm_dataset

,Recency,Frequency,Monetary
customer_id,,,
1,115,11,692985
2,29,11,693777
3,5,8,416376
4,40,9,515838
5,316,4,301383
...,...,...,...
496,35,12,669037
497,25,7,456924
498,25,15,1229418


In [7]:
r = pd.qcut(rfm_dataset['Recency'], 5, labels=[5,4,3,2,1])
f = pd.qcut(rfm_dataset['Frequency'], 5, labels=[1,2,3,4,5])
m = pd.qcut(rfm_dataset['Monetary'], 5, labels=[1,2,3,4,5])
rfm_dataset = rfm_dataset.assign(R=r.values, F=f.values, M=m.values)
rfm_dataset['RFM_Score'] = rfm_dataset[['R','F','M']].sum(axis=1)
rfm_dataset.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score
customer_id,,,,,,,
1,115,11,692985,2,3,4,9
2,29,11,693777,4,3,4,11
3,5,8,416376,5,2,2,9
4,40,9,515838,4,2,2,8
5,316,4,301383,1,1,1,3


In [8]:
def check_status(r, f, m):
    if (r >= 4) and (f >= 4) and (m >= 4):
        return 'VIP'
    elif r >= 3:
        return 'Active'
    elif (r <= 2) and (f >= 3 or m >= 3):
        return 'At Risk'
    else:
        return 'Lost'



In [9]:
rfm_dataset['Customer_Status'] = np.vectorize(check_status)(rfm_dataset['R'], rfm_dataset['F'], rfm_dataset['M'])
rfm_dataset.head(20)

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Customer_Status
customer_id,,,,,,,,
1,115,11,692985,2,3,4,9,At Risk
2,29,11,693777,4,3,4,11,Active
3,5,8,416376,5,2,2,9,Active
4,40,9,515838,4,2,2,8,Active
5,316,4,301383,1,1,1,3,Lost
6,95,7,358224,3,1,1,5,Active
7,30,12,980494,4,4,5,13,VIP
8,113,11,625158,2,3,3,8,At Risk
9,60,13,588789,3,4,3,10,Active


In [10]:
VIP_customers = rfm_dataset[rfm_dataset['Customer_Status'] == 'VIP']
VIP_customers

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Customer_Status
customer_id,,,,,,,,
7,30,12,980494,4,4,5,13,VIP
19,36,12,831868,4,4,5,13,VIP
26,34,18,1238790,4,5,5,14,VIP
29,7,13,677983,5,4,4,13,VIP
34,24,13,1115288,4,4,5,13,VIP
42,42,17,974253,4,5,5,14,VIP
74,7,12,848954,5,4,5,14,VIP
77,5,14,1003235,5,5,5,15,VIP
79,33,16,1130498,4,5,5,14,VIP


In [11]:
active_customers = rfm_dataset[rfm_dataset['Customer_Status'] == 'Active']
active_customers

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Customer_Status
customer_id,,,,,,,,
2,29,11,693777,4,3,4,11,Active
3,5,8,416376,5,2,2,9,Active
4,40,9,515838,4,2,2,8,Active
6,95,7,358224,3,1,1,5,Active
9,60,13,588789,3,4,3,10,Active
...,...,...,...,...,...,...,...,...
489,45,9,655468,4,2,4,10,Active
491,4,7,392447,5,1,2,8,Active
492,91,9,482777,3,2,2,7,Active


In [12]:
customers_at_risk = rfm_dataset[rfm_dataset['Customer_Status'] == 'At Risk']
customers_at_risk

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Customer_Status
customer_id,,,,,,,,
1,115,11,692985,2,3,4,9,At Risk
8,113,11,625158,2,3,3,8,At Risk
11,161,9,645820,2,2,4,8,At Risk
15,235,9,602116,1,2,3,6,At Risk
16,311,16,1159772,1,5,5,11,At Risk
...,...,...,...,...,...,...,...,...
464,129,15,806583,2,5,5,12,At Risk
473,126,9,564819,2,2,3,7,At Risk
476,105,8,526472,2,2,3,7,At Risk


In [13]:
lost_customers = rfm_dataset[rfm_dataset['Customer_Status'] == 'Lost']
lost_customers

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Customer_Status
customer_id,,,,,,,,
5,316,4,301383,1,1,1,3,Lost
14,146,9,310601,2,2,1,5,Lost
17,217,9,509406,1,2,2,5,Lost
33,107,6,369082,2,1,1,4,Lost
36,144,6,281599,2,1,1,4,Lost
...,...,...,...,...,...,...,...,...
475,337,4,187900,1,1,1,3,Lost
490,312,8,425166,1,2,2,5,Lost
494,263,6,239735,1,1,1,3,Lost


In [14]:
segment_summary = rfm_dataset.reset_index().groupby('Customer_Status').agg(
    num_customers = ('customer_id','nunique'),
    avg_recency = ('Recency','mean'),
    avg_frequency = ('Frequency','mean'),
    avg_monetary = ('Monetary','mean'),
    total_revenue = ('Monetary','sum')
).sort_values('total_revenue', ascending=False)
segment_summary

,num_customers,avg_recency,avg_frequency,avg_monetary,total_revenue
Customer_Status,,,,,
Active,240,41.858333,9.575000,558082.866667,133939888
At Risk,121,176.727273,10.958678,706301.694215,85462505
VIP,60,24.950000,14.183333,889162.600000,53349756
Lost,79,240.088608,6.645570,352151.708861,27819985


In [15]:
rfm_dataset.to_csv("../data/rfm_analysis_table.csv")

In [16]:
rfm_dataset.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Score,Customer_Status
customer_id,,,,,,,,
1,115,11,692985,2,3,4,9,At Risk
2,29,11,693777,4,3,4,11,Active
3,5,8,416376,5,2,2,9,Active
4,40,9,515838,4,2,2,8,Active
5,316,4,301383,1,1,1,3,Lost
